In [ ]:
# Preprocessing images and labels for training
# Author: Chengchao Xu
# Date: 2025-03-11

# virtual environment: mini-proj
# "conda activate mini-proj"

# libraries
import os
import numpy as np
from skimage import feature, io, color, exposure, filters
import matplotlib.pyplot as plt


In [ ]:
# TODO: change the image path
# change folder name and file name
# Change folder name and file name
image_dir = os.path.join('..', 'image_files', 'AG56-A')  # Use os.path.join for proper path separators
image_filename = 'AG56-D2-A-50K-002'

# Create input path
test_filepath = os.path.join(image_dir, image_filename +'.tif')

In [ ]:
# Load and preprocess image
test_image = io.imread(test_filepath)
test_image = test_image[0:960, 0:1500]
# Convert to grayscale if necessary
if len(test_image.shape) > 2:
    test_image = color.rgb2gray(test_image)
print(f"Original image shape: {test_image.shape}")

# display the image
plt.imshow(test_image, cmap='gray')
plt.title('Original Image')

# Create output filename for the test image
test_filename = image_filename + '_test.tif'
# Create complete output path
output_path = os.path.join(image_dir, test_filename)

# Save the test image
io.imsave(output_path, test_image.astype(np.uint8))
print(f"Saved test image to: {output_path}")


In [ ]:

# preprocess the image same as training image
from skimage.morphology import disk, white_tophat
selem = disk(50)  # Size controls how large of background variations to remove
test_image = white_tophat(test_image, selem)
image_enhanced = exposure.rescale_intensity(test_image)
image_denoised = filters.gaussian(image_enhanced, sigma=2)

# Display the processed image
image_processed = image_denoised
plt.figure()
plt.imshow(image_processed, cmap='gray')
plt.title('Processed Image')

# Set scale factor, measured using imageJ
SCALE_FACTOR = 0.603 # pixel/nm

# skimage feature extraction
features = feature.multiscale_basic_features(
    image=image_processed,
    intensity=True,
    texture=True,
    edges=True,
    sigma_min=1,
    sigma_max=10,
    channel_axis=None # no color channel
)
print(f"Feature shape: {features.shape}")

In [ ]:


from joblib import load
# load the classifier
pixel_classifier = load('best_pixel_classifier.joblib')
# predict the labels
X = features.reshape(-1, features.shape[2])
predicted_labels = pixel_classifier.predict(X).reshape(image_processed.shape)


In [ ]:

# Display the predicted labels with original test image, sub plot
plt.figure()
plt.subplot(121)
plt.imshow(test_image, cmap='gray')
plt.title('Original Image')
plt.subplot(122)
plt.imshow(predicted_labels, cmap='gray')
plt.title('Predicted Labels')
plt.show()




In [ ]:
# Create output path (with proper naming)
output_filename = image_filename + '_predict_GBs.tif'  # Better naming convention
output_path = os.path.join(image_dir, output_filename)

# Save the predicted labels to the output path
io.imsave(output_path, predicted_labels.astype(np.uint8)) 

print(f"Saved prediction to: {output_path}")
